## Import & read the datasets

In [1]:
import pandas as pd
import numpy as np

In [2]:
# First things first lets load the data
visitor_df = pd.read_csv('visitor_log.csv')
visitor_df.head()

,visit_date,name,dob
0,2016-06-16,Samuel Casanova,1990-03-12
1,2016-06-16,Gerald Orozco,1990-10-05
2,2016-06-16,Sharon Davis,1990-10-18
3,2016-06-16,Spencer Spencer,1991-10-13
4,2016-06-16,Steven Estevez,1992-01-13


In [3]:
visitor_df.drop(columns=["dob"], inplace=True)

In [4]:
visitor_df.shape

(42727, 2)

In [5]:
visitor_df.describe()

,visit_date,name
count,42727,42727
unique,200,985
top,2016-07-19,Willie Gould
freq,246,177


In [6]:
thief_df = pd.read_csv('theft_log.csv', header=None, names=['theft_date'])
thief_df.head()

,theft_date
0,1/1/2017
1,12/30/2016
2,12/25/2016
3,12/22/2016
4,12/19/2016


In [7]:
thief_df.shape

(34, 1)

## Merge the logs

In [8]:
visitor_df.visit_date = pd.to_datetime(visitor_df.visit_date)
thief_df.theft_date = pd.to_datetime(thief_df.theft_date)

In [9]:
merged_log = visitor_df.merge(thief_df, how='left', left_on='visit_date',right_on='theft_date')
merged_log

,visit_date,name,theft_date
0,2016-06-16,Samuel Casanova,NaT
1,2016-06-16,Gerald Orozco,NaT
2,2016-06-16,Sharon Davis,NaT
3,2016-06-16,Spencer Spencer,NaT
4,2016-06-16,Steven Estevez,NaT
...,...,...,...
42722,2017-01-01,Kathryn Dean,2017-01-01
42723,2017-01-01,April Jones,2017-01-01
42724,2017-01-01,Lisa Sing,2017-01-01
42725,2017-01-01,Stephanie Dolphin,2017-01-01


In [10]:
merged_log['theft_bool']= merged_log['theft_date'].apply(lambda x: 0 if pd.isnull(x) else 1)
merged_log

,visit_date,name,theft_date,theft_bool
0,2016-06-16,Samuel Casanova,NaT,0
1,2016-06-16,Gerald Orozco,NaT,0
2,2016-06-16,Sharon Davis,NaT,0
3,2016-06-16,Spencer Spencer,NaT,0
4,2016-06-16,Steven Estevez,NaT,0
...,...,...,...,...
42722,2017-01-01,Kathryn Dean,2017-01-01,1
42723,2017-01-01,April Jones,2017-01-01,1
42724,2017-01-01,Lisa Sing,2017-01-01,1
42725,2017-01-01,Stephanie Dolphin,2017-01-01,1


In [11]:
theft_sum = merged_log.groupby('name').theft_bool.sum().sort_values(ascending=False).reset_index()
theft_sum = theft_sum[theft_sum.theft_bool!=0]
theft_sum.columns = ['name','theft_day_count']
theft_sum

,name,theft_day_count
0,Karen Keeney,30
1,Patricia Whitley,27
2,Christine Victoria,27
3,Judith Sanders,27
4,Henry Brunson,27
...,...,...
901,Theodore Roper,1
902,Joshua Penttila,1
903,Curtis Degroot,1
904,Shirley White,1


In [15]:
not_theft = merged_log[merged_log.theft_bool!=1]

In [16]:
not_theft_count = not_theft.groupby('name').theft_bool.count().sort_values(ascending=False).reset_index()
not_theft_count.columns=['name','not_theft_day_count']
not_theft_count

,name,not_theft_day_count
0,Willie Gould,154
1,Henry Brunson,142
2,Christine Victoria,139
3,Dale Kirkwood,138
4,Arthur Freund,134
...,...,...
977,Hannah Toomey,1
978,Joni Cordova,1
979,Joseph Blanco,1
980,Michelle Karcher,1


In [18]:
final_df = merged_log.merge(not_theft_count, how='left', on='name')
final_df

,visit_date,name,theft_date,theft_bool,not_theft_day_count
0,2016-06-16,Samuel Casanova,NaT,0,38.0
1,2016-06-16,Gerald Orozco,NaT,0,19.0
2,2016-06-16,Sharon Davis,NaT,0,39.0
3,2016-06-16,Spencer Spencer,NaT,0,91.0
4,2016-06-16,Steven Estevez,NaT,0,55.0
...,...,...,...,...,...
42722,2017-01-01,Kathryn Dean,2017-01-01,1,42.0
42723,2017-01-01,April Jones,2017-01-01,1,29.0
42724,2017-01-01,Lisa Sing,2017-01-01,1,62.0
42725,2017-01-01,Stephanie Dolphin,2017-01-01,1,40.0


In [19]:
final_df = final_df.merge(theft_sum, how='left', on='name')
final_df

,visit_date,name,theft_date,theft_bool,not_theft_day_count,theft_day_count
0,2016-06-16,Samuel Casanova,NaT,0,38.0,11.0
1,2016-06-16,Gerald Orozco,NaT,0,19.0,2.0
2,2016-06-16,Sharon Davis,NaT,0,39.0,1.0
3,2016-06-16,Spencer Spencer,NaT,0,91.0,14.0
4,2016-06-16,Steven Estevez,NaT,0,55.0,11.0
...,...,...,...,...,...,...
42722,2017-01-01,Kathryn Dean,2017-01-01,1,42.0,9.0
42723,2017-01-01,April Jones,2017-01-01,1,29.0,3.0
42724,2017-01-01,Lisa Sing,2017-01-01,1,62.0,13.0
42725,2017-01-01,Stephanie Dolphin,2017-01-01,1,40.0,10.0


In [20]:
final_df.drop(columns='theft_date', inplace=True)
final_df

,visit_date,name,theft_bool,not_theft_day_count,theft_day_count
0,2016-06-16,Samuel Casanova,0,38.0,11.0
1,2016-06-16,Gerald Orozco,0,19.0,2.0
2,2016-06-16,Sharon Davis,0,39.0,1.0
3,2016-06-16,Spencer Spencer,0,91.0,14.0
4,2016-06-16,Steven Estevez,0,55.0,11.0
...,...,...,...,...,...
42722,2017-01-01,Kathryn Dean,1,42.0,9.0
42723,2017-01-01,April Jones,1,29.0,3.0
42724,2017-01-01,Lisa Sing,1,62.0,13.0
42725,2017-01-01,Stephanie Dolphin,1,40.0,10.0


In [27]:
final_df[final_df.isnull().any(axis=1)]

,visit_date,name,theft_bool,not_theft_day_count,theft_day_count
144,2016-06-16,Larry Drabant,0,10.0,NaN
496,2016-06-18,Marie Justice,0,5.0,NaN
567,2016-06-18,William Johnson,0,14.0,NaN
722,2016-06-19,Mary Jefferson,0,3.0,NaN
766,2016-06-19,Brittney Fitzpatrick,0,2.0,NaN
...,...,...,...,...,...
42322,2016-12-31,Susan Lee,0,5.0,NaN
42338,2016-12-31,Dale Winship,0,6.0,NaN
42443,2016-12-31,James Cepeda,0,16.0,NaN
42496,2016-12-31,Kristen Kellum,0,6.0,NaN


In [28]:
final_df = final_df.fillna(0)

In [30]:
final_df.isnull().sum()

visit_date             0
name                   0
theft_bool             0
not_theft_day_count    0
theft_day_count        0
dtype: int64

In [32]:
final_df['total_visit'] = final_df.not_theft_day_count + final_df.theft_day_count
final_df

,visit_date,name,theft_bool,not_theft_day_count,theft_day_count,total_visit
0,2016-06-16,Samuel Casanova,0,38.0,11.0,49.0
1,2016-06-16,Gerald Orozco,0,19.0,2.0,21.0
2,2016-06-16,Sharon Davis,0,39.0,1.0,40.0
3,2016-06-16,Spencer Spencer,0,91.0,14.0,105.0
4,2016-06-16,Steven Estevez,0,55.0,11.0,66.0
...,...,...,...,...,...,...
42722,2017-01-01,Kathryn Dean,1,42.0,9.0,51.0
42723,2017-01-01,April Jones,1,29.0,3.0,32.0
42724,2017-01-01,Lisa Sing,1,62.0,13.0,75.0
42725,2017-01-01,Stephanie Dolphin,1,40.0,10.0,50.0


## Bayes Rule => P(A|B) = P(A) * P(B|A) / P(B)
#### P(A|B) : posterior probability
#### P(A) : prior probability
#### P(B|A) : likelihood
#### P(B) : evidence

### T = Is there any theft? (0,1)
### X = X person at the nightclub (number of person)

### p(T|X) = p(X|T) * P(T) / P(X)

p(X|T) likelihood -> Probability of x person is at the nightclub when theft occurs = theft_day_count/34
p(T) prior -> How many days was theft at the nightclub = 34/200
P(X) evidence -> Probability of x person coming to the nightclub 

In [33]:
final_df["probabilty_of_thief"] = ((final_df.theft_day_count/34) * (34/200)) / (final_df.total_visit/200)

In [40]:
final_df.groupby(by="name").agg({'probabilty_of_thief': 'first'}).sort_values("probabilty_of_thief" ,ascending=False).head(20)

,probabilty_of_thief
name,
Oneida Randall,1.000000
Gabriel Kusel,1.000000
Mary Redwine,1.000000
Shirley Ivey,0.750000
Joseph Blanco,0.666667
Martha Holland,0.666667
Loretta Massey,0.588235
Rusty Monterio,0.500000
Eliseo Markham,0.500000
